#### CAUTION !!

・sequence lengthごとにMinMaxScaleをしているため、全体のグラフをプロットすると不自然な形になっている

In [628]:
year = {2017: 0, 2018: 1}
month = {9: 0, 10: 1, 11: 2, 12: 3, 1: 4, 2: 5}
day = {0: 1, 1: 2, 2: 3, 3: 4, 4: 5}
time = {9: 0, 10: 1, 11:2, 12: 3, 13: 4, 14: 5, 15: 6}

In [629]:
a = torch.Tensor(32, 64, 10)

In [636]:
a[:, :, 5].shape

torch.Size([32, 64])

In [633]:
m = nn.Embedding(len(year), 20)
embedded = m(torch.LongTensor([0]))

In [634]:
embedded.shape

torch.Size([1, 20])

In [2]:
import torch
import pandas as pd
import numpy as np

from Stock_Loader import load_data
from Stock_Preprocessor import process, fast_plot
from Stock_RNN import get_model_0

import warnings
warnings.filterwarnings("ignore")

from importlib import reload

import math

In [1130]:
### RELOAD ###

import Stock_Preprocessor
import Stock_Loader
import Stock_RNN

reload(Stock_Preprocessor)
reload(Stock_RNN)
reload(Stock_Loader)

from Stock_Loader import load_data
from Stock_Preprocessor import process, fast_plot
from Stock_RNN import get_model_0

In [3]:
# Global Import

import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import torch.optim as optim

In [5]:
df_appl = load_data("../Datasets/dataset.csv")

...Downloading Datasets
...Adding Values to Datasets


In [613]:
###### DOWNLOADED #######

df_appl.head(15600)

,TimeStamp,Open,High,Low,Close,Volume,Year,Month,Day,Hour,Day_of_Week
0,2017-09-11 09:30:00,160.5,160.62,160.5,160.57,407091.0,2017,9,11,9,0
1,2017-09-11 09:31:00,160.58,160.76,160.51,160.51,266236.0,2017,9,11,9,0
2,2017-09-11 09:32:00,160.525,160.53,160.23,160.32,209863.0,2017,9,11,9,0
3,2017-09-11 09:33:00,160.32,160.36,160.03,160.15,205695.0,2017,9,11,9,0
4,2017-09-11 09:34:00,160.15,160.3,160.1001,160.2799,140139.0,2017,9,11,9,0
...,...,...,...,...,...,...,...,...,...,...,...
15595,2017-11-03 15:16:00,172.68,172.7,172.66,172.666,32607,2017,11,3,15,4
15596,2017-11-03 15:17:00,172.66,172.69,172.62,172.645,37614,2017,11,3,15,4
15597,2017-11-03 15:18:00,172.65,172.65,172.56,172.584,80428,2017,11,3,15,4
15598,2017-11-03 15:19:00,172.58,172.583,172.381,172.45,152298,2017,11,3,15,4


In [8]:
seq_length = 120
feature = ["Close", "Open", "High", "Low"]
val_length = 600
test_length = 600
feature_range=(-1,1)

train, train_label, val, val_label, test, test_label, meta, scaler =\
                                            process(df_appl, feature, feature_range, seq_length, val_length, test_length)

...186 row(s) dropped
...Transforming Datasets
...Cleaning Datasets
...Done !


In [293]:
df_appl.iloc[42364:42366, 0:5] # 2018-02-15 -> apple revenue hit all time hight

,TimeStamp,Open,High,Low,Close
42364,2018-02-14 16:00:00,167.32,167.45,167.3,167.37
42365,2018-02-15 09:30:00,169.79,169.86,169.36,169.84


In [9]:
df_check = pd.DataFrame(train[0])
df_check.head(5)

,0,1,2,3
0,-1.000000,-0.970443,-1.000000,-0.945595
1,-0.769163,-1.000000,-0.852258,-0.945595
2,-0.614103,-0.759324,-0.677419,-0.711403
3,-0.929837,-0.618578,-0.677419,-0.837507
4,-0.493071,-0.952850,-0.580645,-0.869933


In [10]:
df_checked = pd.DataFrame(scaler.inverse_transform(train[0]))
df_checked.head(5)

,0,1,2,3
0,172.500000,172.518025,172.500000,172.533179
1,172.640776,172.500000,172.590100,172.533179
2,172.735339,172.646776,172.696726,172.676001
3,172.542789,172.732610,172.696726,172.599097
4,172.809150,172.528754,172.755744,172.579321


In [576]:
input_size = 4
output_size = 4
hidden_dim = 128
n_layers = 2
bidirectional = False
dropout = 0.1
lr = 0.001
scheduler_step = [75, 100]
gamma = 0.1
log_dir= "stock_directional_loss"

model, optimizer, mse_loss, scheduler, writer = get_model_0(input_size, output_size, hidden_dim, n_layers, bidirectional, dropout,
                                                             lr, scheduler_step, gamma, log_dir)

total_loss = DirectionalMSELoss()

In [577]:
writer = SummaryWriter(log_dir)

In [578]:
num_epochs = 150
bt_size = 64
acc = math.inf

best_acc = math.inf
best_model = model
loss_accumulated = []
sample = []

In [579]:
for epoch in range(num_epochs):
    
    for phase in ["train", "val"]:
        
        if phase == "train":
            
            model.train()
            
            for i in range(0, len(train)-1, bt_size):
                
                optimizer.zero_grad()
                
                inputs, targets = train[i:i+bt_size, :, :], train_label[i:i+bt_size, :]
                
                if inputs.shape[0] < bt_size: break
                
                inputs = torch.from_numpy(inputs).float()
                targets = torch.from_numpy(targets).float()

                outputs = model(inputs)

                loss = total_loss(inputs, targets, outputs)
                loss_accumulated.append(loss.item()*1000)
                
                loss.backward(retain_graph=True)
                
                torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
                
                if (i % 3600 == 0) and (i > bt_size):
                    acc = sum(loss_accumulated)*(100) / (len(loss_accumulated))
                    print("....Processing {}th sample out of {} training set. Loss: {}".format(i+1,
                                                                                             len(train), acc))
                    writer.add_scalar("Train_Loss", acc,
                                                      (epoch)*(len(train) // 3600) + (i // 3600))
                    
                optimizer.step()
                scheduler.step()
                
            acc = sum(loss_accumulated)*(100) / (len(loss_accumulated))

            
            print("Epoch: {} Loss: {:.3f}".format(epoch, acc))
            
            loss_accumulated = []
            
            perm = np.random.permutation(train.shape[0])
            train, train_label = train[perm], train_label[perm]
            
        elif phase == "val":
            
            print("...Evaluation Mode")
            
            model.eval()
            
            with torch.no_grad():
                
                for i in range(0, len(val)-1, bt_size):

                    inputs, targets = val[i:i+bt_size,:,:], val_label[i:i+bt_size, :]
                    
                    if inputs.shape[0] < bt_size: break
                    
                    
                    inputs = torch.from_numpy(inputs).float()
                    targets = torch.from_numpy(targets).float()
                    
                    outputs = model(inputs)
                    
                    loss = loss_func(outputs.view(-1, 4), targets)
                    loss_accumulated.append(loss.item()*1000)
                
                acc = sum(loss_accumulated)*(100) / (len(loss_accumulated))

                writer.add_scalar("Val_Loss", acc, epoch)
                
                print("Validation: {:.3f}".format(acc))
                
                if acc < best_acc:
                    best_model = model
                    best_acc = acc
                    print("MODEL UPDATED !!")
                
                loss_accumulated = []
                
                perm = np.random.permutation(val.shape[0])
                val, val_label = val[perm], val_label[perm]
                
    print("-" * 70)
    print()

....Processing 14401th sample out of 41640 training set. Loss: 7827.628876980954
....Processing 28801th sample out of 41640 training set. Loss: 6766.863643065789
Epoch: 0 Loss: 6345.291
...Evaluation Mode
Validation: 1832.879
MODEL UPDATED !!
----------------------------------------------------------------------

....Processing 14401th sample out of 41640 training set. Loss: 5518.672771883749
....Processing 28801th sample out of 41640 training set. Loss: 5465.53082797189
Epoch: 1 Loss: 5330.906
...Evaluation Mode
Validation: 1633.276
MODEL UPDATED !!
----------------------------------------------------------------------

....Processing 14401th sample out of 41640 training set. Loss: 5279.111118772916
....Processing 28801th sample out of 41640 training set. Loss: 5087.132569394196
Epoch: 2 Loss: 5026.432
...Evaluation Mode
Validation: 1482.203
MODEL UPDATED !!
----------------------------------------------------------------------

....Processing 14401th sample out of 41640 training set.

Validation: 848.521
----------------------------------------------------------------------

....Processing 14401th sample out of 41640 training set. Loss: 3622.4964742375687
....Processing 28801th sample out of 41640 training set. Loss: 3573.8821754731784
Epoch: 27 Loss: 3606.168
...Evaluation Mode
Validation: 864.807
----------------------------------------------------------------------

....Processing 14401th sample out of 41640 training set. Loss: 3570.553827701153
....Processing 28801th sample out of 41640 training set. Loss: 3546.1223000730747
Epoch: 28 Loss: 3601.297
...Evaluation Mode
Validation: 837.447
----------------------------------------------------------------------

....Processing 14401th sample out of 41640 training set. Loss: 3554.7838117762476
....Processing 28801th sample out of 41640 training set. Loss: 3547.8156163231233
Epoch: 29 Loss: 3589.323
...Evaluation Mode
Validation: 851.138
----------------------------------------------------------------------

....Proce

....Processing 28801th sample out of 41640 training set. Loss: 3497.1668647226365
Epoch: 54 Loss: 3484.153
...Evaluation Mode
Validation: 830.813
----------------------------------------------------------------------

....Processing 14401th sample out of 41640 training set. Loss: 3534.49967698054
....Processing 28801th sample out of 41640 training set. Loss: 3529.0080731829626
Epoch: 55 Loss: 3477.768
...Evaluation Mode
Validation: 790.600
----------------------------------------------------------------------

....Processing 14401th sample out of 41640 training set. Loss: 3541.1972272316966
....Processing 28801th sample out of 41640 training set. Loss: 3535.7743774245423
Epoch: 56 Loss: 3481.185
...Evaluation Mode
Validation: 817.689
----------------------------------------------------------------------

....Processing 14401th sample out of 41640 training set. Loss: 3405.8846351979055
....Processing 28801th sample out of 41640 training set. Loss: 3450.243330732021
Epoch: 57 Loss: 3476.

KeyboardInterrupt: 

In [322]:
def find_outlier(df, column):
    idx_list = list()
    prev = df[column].iloc[0]
    for idx, row in enumerate(df[column]):
        if abs(float(row) - float(prev)) > 1:
            idx_list.append(idx)
        prev = row
    return idx_list

In [582]:
def result_compare(model, df, df_label, idx_list):
    def compare_and_print(df, df_label, idx):
            subject = torch.from_numpy(df[idx:idx+1, :, :])
            predict = (subject).float()
            predicted = model(predict)
            target = df_label[idx, :]
            previous = df[idx:idx+1, -1, :][0]
            print("-"*70)
            print("idx: {:3}".format(idx))
            print("                                   \t{}".format(feature))
            print()
            print("Predicted(y_pred)  :\t{}".format(predicted.detach().numpy()[0]))
            print("Target(y^)               :\t{}".format(target))
            print()
            print("Last Price(y-1)        :\t{}".format(previous))
            print()
            print("(y^ - (y_pred))      => {}".format((predicted - torch.from_numpy(target).float()).abs().sum()))
            print("(y^ - (y-1))             => {}".format((torch.from_numpy(target).float() - torch.from_numpy(previous).float()).abs().sum()))
            print("-"*70)
    if type(idx_list) == int:
        compare_and_print(df, df_label, idx_list)
    elif type(idx_list) == list:
        for idx in idx_list:
            compare_and_print(df, df_label, idx)

In [583]:
result_compare(best_model, test, test_label, [i for i in range(10, 20)])

----------------------------------------------------------------------
idx:  10
                                   	['Close', 'Open', 'High', 'Low']

Predicted(y_pred)  :	[0.6992865 0.689968  0.6791362 0.688267 ]
Target(y^)               :	[0.88981132 0.78787879 0.83870968 0.83458647]

Last Price(y-1)        :	[0.77358491 0.84848485 0.80645161 0.63909774]

(y^ - (y_pred))      => 0.5943285226821899
(y^ - (y-1))             => 0.404579222202301
----------------------------------------------------------------------
----------------------------------------------------------------------
idx:  11
                                   	['Close', 'Open', 'High', 'Low']

Predicted(y_pred)  :	[0.8782132  0.89291906 0.88660765 0.89947975]
Target(y^)               :	[0.8490566  0.89393939 0.83870968 0.83458647]

Last Price(y-1)        :	[0.88981132 0.78787879 0.83870968 0.83458647]

(y^ - (y_pred))      => 0.14296823740005493
(y^ - (y-1))             => 0.14681529998779297
--------------------------

In [575]:
class DirectionalMSELoss(nn.Module):
    def __init__(self):
        super(DirectionalMSELoss, self).__init__()
    
    def forward(self, inputs, label, outputs, scale=1e-4):

        true_dif = inputs[:, -1, :] - label[:, :] 
        pred_dif = inputs[:, -1, :] - outputs[:, :]
        dif_rel = torch.abs(a - b) # 傾きの差 (relative)

        #         epsilon = 1e-6
        #         dif_abs = (a + epsilon) * (b + epsilon)  傾きのサインの差 if negative then + and - (absolute)

        dif_sum = torch.sum(dif_rel)
        dif_sum_scaled = dif_sum * scale
        
        mse =  mse_loss(outputs.view(-1, 4), label)
        
        return dif_sum_scaled + mse

In [581]:
# torch.save(best_model, "stock_model_v4")

In [332]:
# calculate baseline (not working)

def mse_copy_last_baseline(df, df_label, bt_size, scale=100):
    mse = []
    for i in range(df.shape[0]):
        predicted = df[i][-1]
        target = df_label[i]
        loss = ((predicted - target) **2).sum()
        mse.append(loss)
    return scale * (sum(mse) / (len(mse) // bt_size))

In [333]:
mse_copy_last_baseline(train, train_label, bt_size)

558.2576291834196

### Validation

分散された位置からとってきた方がいい（今のままだと連続したところからとってきている）

In [1]:
###########################################
###STEP 1 まずEmbeddingなしでClose Priceだけで予測する##
###########################################

In [2]:
##########################################
############STEP 2 Price4つを加える#############
##########################################

In [3]:
##########################################
###########STEP 3 Parametersをいじくる###########
##########################################

In [ ]:
##########################################
##########STEP 4 Embeddingsを加える#############
##########################################

In [ ]:
##########################################
###########STEP 6 Loss Functionを変える##########
##########################################

In [4]:
##########################################
########STEP 5 Transformerを使って予測する#########
##########################################

In [7]:
##########################################
########STEP 7 Company Embeddingを加える########
##########################################

In [8]:
##########################################
########STEP 8 Day, Hour, MinuteをMixする##########
##########################################